# Modeling

## From the end of EDA:

### Conclusion

So the moral of the story currently is that we have at the minimum a couple of heuristics for choosing players:

- Choose value players, ie players with moderate price tags but good matchups
- Choose players based on Def they play
- Avoid expensive players, since statistically they are unable to produce high scores consistently.

With these guidelines, week 1 will be a total gamble, since we won't have any real data besides salaries. Week 2 will be the first time we can use any defensive data to help with our decision making.

## Goal for this notebook:

Based on the conclusions from the EDA, we want to see if we can find a model that confirms these ideas across seasons, and also has a high enough (cross-validated) accuracy to warrant trying to use this with real money.

### Note:
Sci-kit Learn says, according to https://scikit-learn.org/stable/tutorial/machine_learning_map/, that we should be using the linear SVC classifier, but for the sake of this exercise, we are going to try many different models to see what produces the best result.

## Logic

Instead of projecting individual player points, the notebook is going to classify players based on their potentials to score in certain catgories.

- A player in the 0 category will be likely to score 15 points or less (players that should be ignored).
- A player with a 1 classification will be likely to score between 15 and 20 points.
- A player with a 2 classification will be likely to score 20+ points.
- A player with a 3 classification will be likely to score 30+ points.

Obviously we want to get as many true 3s as possible, but getting 100% accuracy on that seems implausible. So our model should tend to maximize the top left value (correctly predict poor picks) and have errors that trend towards the bottom right (bottom right 2x2) of the confusion matrix. The model should also minimize the rest of the values on the top row, and the left column.

So the criteria for deciding on what model to proceed with is (in order of importance):
1. Correct 3 predictions
2. Correct 0 predictions
3. Bottom right 2x2 has most counts
4. Minimize top row 
5. Minimize left column

### Jump to:

- [Model Testing](#test_run)
- [Lineup Builder](#lineup_builder)

## Import Libraries

In [1]:
import sys
# !conda install --yes --prefix {sys.prefix} -c conda-forge scikit-learn

In [2]:
from collections import defaultdict
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None # turn off some warnings
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

from xgboost import XGBClassifier

In [3]:
# need version 0.24.2 of sci-kit learn for this notebook to work
# import sklearn
# print(sklearn.__version__)

## Helper Functions

In [4]:
def get_weekly_data(week, year):
    """ get player data for designated week """
    file_path = f"./csv's/{year}/year-{year}-week-{week}-DK-player_data.csv"
    df = pd.read_csv(file_path)
    return df

def get_ytd_season_data(year, current_week):
    """ get data for current season up to most recent week """
    df = get_weekly_data(1,year)
    for week in range(2,current_week+1):
        try:
            df = df.append(get_weekly_data(week, year), ignore_index=True)
        except:
            print("No data for week: "+str(week))
    df = df.drop(['Unnamed: 0', 'Year'], axis=1)
    return df

def get_season_data(year):
    """ get entire season of data """
    df = get_weekly_data(1,year)
    for week in range(2,17):
        try:
            df = df.append(get_weekly_data(week, year), ignore_index=True)
        except:
            print("No data for week: "+str(week))
    df = df.drop(['Unnamed: 0', 'Year'], axis=1)
    return df

def make_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    acc_score = accuracy_score(y_test, y_pred)
    return cm, acc_score

def scale_features(sc_salary, sc_points, sc_pts_ald, X_train, X_test, first_time=False):
    """ scales data for training """
    if first_time:
        X_train['DK salary'] = sc_salary.fit_transform(X_train['DK salary'].values.reshape(-1,1))
        X_train['DK points'] = sc_points.fit_transform(X_train['DK points'].values.reshape(-1,1))
        X_train['Oppt_pts_allowed_lw'] = sc_pts_ald.fit_transform(X_train['Oppt_pts_allowed_lw'].values.reshape(-1,1))
    X_test['DK salary'] = sc_salary.transform(X_test['DK salary'].values.reshape(-1,1))
    X_test['DK points'] = sc_points.transform(X_test['DK points'].values.reshape(-1,1))
    X_test['Oppt_pts_allowed_lw'] = sc_pts_ald.transform(X_test['Oppt_pts_allowed_lw'].values.reshape(-1,1))
    return X_train, X_test

def unscale_features(sc_salary, sc_points, sc_pts_ald, X_train, X_test):
    """ used to change features back so that human readable information can be used to assess
    lineups and player information and performance"""
    X_train['DK salary'] = sc_salary.inverse_transform(X_train['DK salary'].values.reshape(-1,1))
    X_train['DK points'] = sc_points.inverse_transform(X_train['DK points'].values.reshape(-1,1))
    X_train['Oppt_pts_allowed_lw'] = sc_pts_ald.inverse_transform(X_train['Oppt_pts_allowed_lw'].values.reshape(-1,1))
    X_test['DK salary'] = sc_salary.inverse_transform(X_test['DK salary'].values.reshape(-1,1))
    X_test['avg_points'] = sc_points.inverse_transform(X_test['avg_points'].values.reshape(-1,1))
    X_test['Oppt_pts_allowed_lw'] = sc_pts_ald.inverse_transform(X_test['Oppt_pts_allowed_lw'].values.reshape(-1,1))
    return X_train, X_test

def find_15_ptrs(df):
    df['scoring_potential'] = 0
    df['scoring_potential'] = np.where(df['DK points'] >= 10.0, 1, df['scoring_potential'])
    return df

def find_20_ptrs(df):
    df['scoring_potential'] = np.where(df['DK points'] >= 20.0, 2, df['scoring_potential'])
    return df

def find_30_ptrs(df):
    df['scoring_potential'] = np.where(df['DK points'] >= 30.0, 3, df['scoring_potential'])
    return df

def find_scoring_potentials(df):
    """ classifies players as low, med, or high potentials for scoring points """
    df = find_15_ptrs(df)
    df = find_20_ptrs(df)
    df = find_30_ptrs(df)
    return df

def handle_nulls(df):
    """ players that have nulls for any of the columns are 
    extremely likely to be under performing or going into a bye.
    the one caveat is that some are possibly coming off a bye.
    to handle this later, probably will drop them, save those
    as a variable, and then re-merge after getting rid of the other
    null values. """
    df = df.dropna()
    return df

## Import Data

In [5]:
season = 2019
dataset = get_season_data(season)
# dataset

In [6]:
df = handle_nulls(dataset)
df = find_scoring_potentials(df)
df

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,scoring_potential
0,1,"Jackson, Lamar",QB,bal,a,mia,36.56,6000,3
1,1,"Prescott, Dak",QB,dal,h,nyg,36.40,5900,3
2,1,"Watson, Deshaun",QB,hou,a,nor,31.72,6800,3
3,1,"Stafford, Matthew",QB,det,a,ari,31.60,5400,3
4,1,"Mahomes II, Patrick",QB,kan,a,jac,30.32,7200,3
...,...,...,...,...,...,...,...,...,...
6398,16,Cincinnati,Def,cin,a,mia,0.00,2900,0
6399,16,Carolina,Def,car,a,ind,-1.00,2400,0
6400,16,Washington,Def,was,h,nyg,-1.00,2800,0
6401,16,New York G,Def,nyg,a,was,-1.00,2800,0


In [7]:
def_df = df.loc[df.Pos == 'Def']
def_df

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,scoring_potential
414,1,San Francisco,Def,sfo,a,tam,27.0,2200,2
415,1,Tennessee,Def,ten,a,cle,23.0,2600,2
416,1,New York J,Def,nyj,h,buf,18.0,3100,1
417,1,Minnesota,Def,min,h,atl,16.0,3300,1
418,1,Green Bay,Def,gnb,a,chi,14.0,2700,1
...,...,...,...,...,...,...,...,...,...
6398,16,Cincinnati,Def,cin,a,mia,0.0,2900,0
6399,16,Carolina,Def,car,a,ind,-1.0,2400,0
6400,16,Washington,Def,was,h,nyg,-1.0,2800,0
6401,16,New York G,Def,nyg,a,was,-1.0,2800,0


In [8]:
# isolate defenses and assess how many fantasy 
# points they allowed last week. Then add that 
# as a feature to the training data. The idea is
# the defenses that consistently allow the most points
# will also product the highest scoring players

def_df['fantasy_points_allowed_lw'] = 0
df['Oppt_pts_allowed_lw'] = 0
def_teams = [x for x in def_df['Team'].unique()]

for week in range(1,17):
    for team in def_teams:
        try:
            offense_df1 = df.loc[(df['Oppt']==team)&(df['Week']==week)]
            offense_df2 = df.loc[(df['Oppt']==team)&(df['Week']==week+1)]
            sum_ = offense_df1['DK points'].sum()
            def_df.loc[(df['Team']==team)&(df['Week']==week+1), 'fantasy_points_allowed_lw'] = sum_
            df.loc[(df['Oppt']==team)&(df['Week']==week+1), 'Oppt_pts_allowed_lw'] = sum_
        except:
            print('couldnt append data')
            pass

In [9]:
def_df

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,scoring_potential,fantasy_points_allowed_lw
414,1,San Francisco,Def,sfo,a,tam,27.0,2200,2,0.00
415,1,Tennessee,Def,ten,a,cle,23.0,2600,2,0.00
416,1,New York J,Def,nyj,h,buf,18.0,3100,1,0.00
417,1,Minnesota,Def,min,h,atl,16.0,3300,1,0.00
418,1,Green Bay,Def,gnb,a,chi,14.0,2700,1,0.00
...,...,...,...,...,...,...,...,...,...,...
6398,16,Cincinnati,Def,cin,a,mia,0.0,2900,0,96.42
6399,16,Carolina,Def,car,a,ind,-1.0,2400,0,119.44
6400,16,Washington,Def,was,h,nyg,-1.0,2800,0,128.94
6401,16,New York G,Def,nyg,a,was,-1.0,2800,0,99.26


In [10]:
df = df[df.Week != 1]
df

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,scoring_potential,Oppt_pts_allowed_lw
446,2,"Mahomes II, Patrick",QB,kan,a,oak,35.62,7500,3,81.02
447,2,"Jackson, Lamar",QB,bal,h,ari,33.88,6700,3,137.50
448,2,"Prescott, Dak",QB,dal,a,was,28.66,6300,2,129.12
449,2,"Wilson, Russell",QB,sea,a,pit,28.20,6200,2,130.12
450,2,"Ryan, Matt",QB,atl,h,phi,25.10,6100,2,122.00
...,...,...,...,...,...,...,...,...,...,...
6398,16,Cincinnati,Def,cin,a,mia,0.00,2900,0,123.56
6399,16,Carolina,Def,car,a,ind,-1.00,2400,0,134.68
6400,16,Washington,Def,was,h,nyg,-1.00,2800,0,99.26
6401,16,New York G,Def,nyg,a,was,-1.00,2800,0,128.94


In [11]:
X = df.drop(labels='scoring_potential', axis=1)
y = df['scoring_potential']

In [12]:
# ordinal encode names, teams, h/a and oppts
oe_names = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)
oe_teams = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)
oe_ha = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)
oe_oppts = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

def ft_ordinal_encode_df(df, oe_names, oe_teams, oe_ha, oe_oppts):
    df['Name'] = oe_names.fit_transform(df['Name'].values.reshape(-1,1))
    df['Team'] = oe_teams.fit_transform(df['Team'].values.reshape(-1,1))
    df['Oppt'] = oe_oppts.fit_transform(df['Oppt'].values.reshape(-1,1))
    df['h/a'] = oe_ha.fit_transform(df['h/a'].values.reshape(-1,1))
    return df

def t_ordinal_encode_df(df, oe_names, oe_teams, oe_ha, oe_oppts):
    df['Name'] = oe_names.transform(df['Name'].values.reshape(-1,1))
    df['Team'] = oe_teams.transform(df['Team'].values.reshape(-1,1))
    df['Oppt'] = oe_oppts.transform(df['Oppt'].values.reshape(-1,1))
    df['h/a'] = oe_ha.transform(df['h/a'].values.reshape(-1,1))
    return df

def it_ordinal_encode_df(df, oe_names, oe_teams, oe_ha, oe_oppts):
    df['Name'] = oe_names.inverse_transform(df['Name'].values.reshape(-1,1))
    df['Team'] = oe_teams.inverse_transform(df['Team'].values.reshape(-1,1))
    df['Oppt'] = oe_oppts.inverse_transform(df['Oppt'].values.reshape(-1,1))
    df['h/a'] = oe_ha.inverse_transform(df['h/a'].values.reshape(-1,1))
    return df

X = ft_ordinal_encode_df(X, oe_names, oe_teams, oe_ha, oe_oppts)

In [13]:
# one hot encode the positions
X = pd.get_dummies(X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
xtr_cols = X_train.columns
xte_cols = X_test.columns

In [15]:
data_to_use = 'scaled'
# data_to_use = 'un-scaled' # comment out this line for using scaled data

In [16]:
if data_to_use == 'scaled':
#     sc_salary = StandardScaler()
#     sc_points = StandardScaler()
#     sc_pts_ald = StandardScaler()
    sc_salary = MinMaxScaler()
    sc_points = MinMaxScaler()
    sc_pts_ald = MinMaxScaler()
    X_train, X_test = scale_features(sc_salary, sc_points, sc_pts_ald, X_train, X_test, first_time=True)

## Non-Boost Methods

Test some models to see which have the best performance overall based on an entire previous season of data. Based on that, use a model or 2 to predict outcomes on a different season, where we will use a partial season of data to predict the next week in sequence, much like would happen in a live situation.

In [17]:
best_acc_method = ""
best_f1_method = ""
best_acc = -100
best_f1 = -100

# Logistic Regression
def make_log_reg(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

lr_acc = 0
try:
    cm, acc_score, f1 = make_log_reg(X_train, y_train,X_test,y_test)
    lr_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "Logistic Regression"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "Logistic Regression"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[836   0   0   2]
 [236   3   0   0]
 [ 83   4   0   0]
 [ 27   1   0   0]]


Accuracy: 

0.7038590604026845


F1: 

0.5867763013292878


C:\Users\maxw2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
# K-NN 
def make_knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

knn_acc = 0
try:
    cm, acc_score, f1 = make_knn(X_train, y_train,X_test,y_test)
    knn_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "K-NN"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "K-NN"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[735  88  13   2]
 [190  40   8   1]
 [ 59  22   2   4]
 [ 20   6   2   0]]


Accuracy: 

0.6518456375838926


F1: 

0.60425706470805


In [19]:
# SVM 
def make_svm(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

svm_acc = 0
try:
    cm, acc_score, f1 = make_svm(X_train, y_train,X_test,y_test)
    svm_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "SVM"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "SVM"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[834   4   0   0]
 [ 12 227   0   0]
 [  0  11  76   0]
 [  0   0   7  21]]


Accuracy: 

0.9714765100671141


F1: 

0.9709819124403616


In [20]:
# Kernel SVM

def make_k_svm(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

k_svm_acc = 0
try:
    cm, acc_score, f1 = make_k_svm(X_train, y_train,X_test,y_test)
    k_svm_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "Kernel SVM"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "Kernel SVM"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[838   0   0   0]
 [239   0   0   0]
 [ 87   0   0   0]
 [ 28   0   0   0]]


Accuracy: 

0.7030201342281879


F1: 

0.5804245049095778


In [21]:
# Naive Bayes
def make_nb(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

nb_acc = 0
try:
    cm, acc_score, f1 = make_nb(X_train, y_train,X_test,y_test)
    nb_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "Naive Bayes"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "Naive Bayes"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[798  40   0   0]
 [ 12 222   5   0]
 [  0   5  66  16]
 [  0   0   2  26]]


Accuracy: 

0.9328859060402684


F1: 

0.9344368015494524


In [22]:
# Decision Tree 
def make_tree(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

dt_acc = 0
try:
    cm, acc_score, f1 = make_tree(X_train, y_train,X_test,y_test)
    dt_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "Decision Tree"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "Decision Tree"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
    pass
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[837   1   0   0]
 [  0 239   0   0]
 [  0   0  87   0]
 [  0   0   0  28]]


Accuracy: 

0.9991610738255033


F1: 

0.9991616991055986


In [23]:
# Random Forest
def make_forest(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

rf_acc = 0
try:
    cm, acc_score, f1 = make_forest(X_train, y_train,X_test,y_test)
    rf_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "Random Forest"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "Random Forest"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")


Confusion Matrix: 

[[837   1   0   0]
 [  0 239   0   0]
 [  0   5  79   3]
 [  0   0   5  23]]


Accuracy: 

0.988255033557047


F1: 

0.9880660304860023


In [24]:
# Summary
print(best_acc)
print(best_acc_method)
print(best_f1)
print(best_f1_method)

0.9991610738255033
Decision Tree
0.9991616991055986
Decision Tree


## Boost Methods (using non-scaled data)

In [25]:
best_acc_method = ""
best_f1_method = ""
best_acc = -100
best_f1 = -100

# AdaBoost
def make_adaboost(X_train, y_train, X_test, y_test):
    classifier = AdaBoostClassifier()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

ada_acc = 0
try:
    cm, acc_score, f1 = make_adaboost(X_train, y_train,X_test,y_test)
    ada_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "AdaBoost"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "AdaBoost"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[837   1   0   0]
 [  0 239   0   0]
 [  0  87   0   0]
 [  0   0   0  28]]


Accuracy: 

0.9261744966442953


F1: 

0.8954200426944888


In [26]:
# GradientBoost
def make_gradientboost(X_train, y_train, X_test, y_test):
    classifier = GradientBoostingClassifier()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

grad_acc = 0
try:
    cm, acc_score, f1 = make_gradientboost(X_train, y_train,X_test,y_test)
    grad_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "Gradient Boost"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "Gradient Boost"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

Confusion Matrix: 

[[837   1   0   0]
 [  0 239   0   0]
 [  0   0  87   0]
 [  0   0   0  28]]


Accuracy: 

0.9991610738255033


F1: 

0.9991616991055986


In [27]:
# XGBoost
def make_xgboost(X_train, y_train, X_test, y_test):
    classifier = XGBClassifier()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm, acc_score = make_confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Confusion Matrix: \n")
    print(cm)
    print("\n")
    print("Accuracy: \n")
    print(acc_score)
    print("\n")
    print("F1: \n")
    print(f1)
    return cm, acc_score, f1

xg_acc = 0
try:
    cm, acc_score, f1 = make_xgboost(X_train, y_train,X_test,y_test)
    xg_acc = acc_score
    if acc_score > best_acc:
        best_acc = acc_score
        best_acc_method = "XGBoost"
    if f1 > best_f1:
        best_f1 = f1
        best_f1_method = "XGBoost"
except ValueError:
    # sample sizes are mismatched
    print("ValueError")
except IndexError:
    # end of the loop
    print("IndexError")

[10:44:14] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\maxw2\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Confusion Matrix: 

[[837   1   0   0]
 [  0 239   0   0]
 [  0   0  87   0]
 [  0   0   0  28]]


Accuracy: 

0.9991610738255033


F1: 

0.9991616991055986


In [28]:
# Summary
print(best_acc)
print(best_acc_method)
print(best_f1)
print(best_f1_method)

0.9991610738255033
Gradient Boost
0.9991616991055986
Gradient Boost


## Results

In [29]:
print("LR: " + str(lr_acc))
print("KNN: " + str(knn_acc))
print("SVM: " + str(svm_acc))
print("K_SVM: " + str(k_svm_acc))
print("NB: " + str(nb_acc))
print("DT: " + str(dt_acc))
print("RF: " + str(rf_acc))
print("Ada: " + str(ada_acc))
print("Grad: " + str(grad_acc))
print("XGB: " + str(xg_acc))

LR: 0.7038590604026845
KNN: 0.6518456375838926
SVM: 0.9714765100671141
K_SVM: 0.7030201342281879
NB: 0.9328859060402684
DT: 0.9991610738255033
RF: 0.988255033557047
Ada: 0.9261744966442953
Grad: 0.9991610738255033
XGB: 0.9991610738255033


From just the baselines, all of the models have super high mean accuracies, which is promising, but I remain skeptical. SVM, Naive Bayes, and Decision Trees are at the top for non-boosted methods, while Gradient and XGBoosting are the best boosted methods.

So next, we want to try a run where we use an example week of data, and try to project possible high scoring players for the following week.

We'll use the best scoring methods mentioned above to see how they perform.

## Test Run

<a id='test_run'></a>

In [30]:
# let's say that week 7 just finished, and week 8 is coming up.
# time to put together a lineup for week 8, or at least form a pool
# of suspected high scoring players.

# mess with these numbers to experiment, but 
# comments below are for week 7 in the 2020 season

season = 2020
week = 7
next_week = week+1 # will be used a little later

if week == 1:
    dataset = get_season_data(season-1)
else: 
    dataset = get_ytd_season_data(season, week)
    
df_next_week = get_weekly_data(next_week, season).drop(labels=['Unnamed: 0', 'Year'], axis=1)

In [31]:
dataset

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary
0,1,"Wilson, Russell",QB,sea,a,atl,34.78,7000
1,1,"Rodgers, Aaron",QB,gnb,a,min,33.76,6300
2,1,"Allen, Josh",QB,buf,h,nyj,33.18,6500
3,1,"Ryan, Matt",QB,atl,h,sea,27.90,6700
4,1,"Jackson, Lamar",QB,bal,h,cle,27.50,8100
...,...,...,...,...,...,...,...,...
2857,7,New Orleans,Def,nor,h,car,1.00,3200
2858,7,Seattle,Def,sea,a,ari,0.00,2700
2859,7,Cincinnati,Def,cin,h,cle,-2.00,2800
2860,7,Houston,Def,hou,h,gnb,-2.00,2200


In [32]:
df_next_week

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary
0,8,"Mahomes II, Patrick",QB,kan,h,nyj,39.64,8100
1,8,"Wilson, Russell",QB,sea,h,sfo,28.74,7800
2,8,"Stafford, Matthew",QB,det,h,ind,27.44,6400
3,8,"Rodgers, Aaron",QB,gnb,h,min,23.54,7600
4,8,"Herbert, Justin",QB,lac,h,den,23.22,6900
...,...,...,...,...,...,...,...,...
378,8,Detroit,Def,det,h,ind,0.00,2500
379,8,Green Bay,Def,gnb,h,min,0.00,2900
380,8,Tennessee,Def,ten,a,cin,-1.00,3000
381,8,San Francisco,Def,sfo,a,sea,-2.00,2700


In [33]:
dataset = dataset.fillna(0)

In [34]:
df_ytd = find_scoring_potentials(dataset)
df_ytd

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,scoring_potential
0,1,"Wilson, Russell",QB,sea,a,atl,34.78,7000,3
1,1,"Rodgers, Aaron",QB,gnb,a,min,33.76,6300,3
2,1,"Allen, Josh",QB,buf,h,nyj,33.18,6500,3
3,1,"Ryan, Matt",QB,atl,h,sea,27.90,6700,2
4,1,"Jackson, Lamar",QB,bal,h,cle,27.50,8100,2
...,...,...,...,...,...,...,...,...,...
2857,7,New Orleans,Def,nor,h,car,1.00,3200,0
2858,7,Seattle,Def,sea,a,ari,0.00,2700,0
2859,7,Cincinnati,Def,cin,h,cle,-2.00,2800,0
2860,7,Houston,Def,hou,h,gnb,-2.00,2200,0


In [35]:
def_df = df_ytd.loc[df_ytd.Pos == 'Def']
def_df

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,scoring_potential
410,1,New Orleans,Def,nor,h,tam,17.0,2400,1
411,1,Washington,Def,was,h,phi,15.0,2000,1
412,1,Baltimore,Def,bal,h,cle,15.0,3100,1
413,1,New England,Def,nwe,h,mia,11.0,3200,1
414,1,LA Chargers,Def,lac,a,cin,11.0,2800,1
...,...,...,...,...,...,...,...,...,...
2857,7,New Orleans,Def,nor,h,car,1.0,3200,0
2858,7,Seattle,Def,sea,a,ari,0.0,2700,0
2859,7,Cincinnati,Def,cin,h,cle,-2.0,2800,0
2860,7,Houston,Def,hou,h,gnb,-2.0,2200,0


In [36]:
def_df['fantasy_points_allowed_lw'] = 0
df_ytd['Oppt_pts_allowed_lw'] = 0
def_teams = [x for x in def_df['Team'].unique()]

for week in range(1,week+1):
    for team in def_teams:
        try:
            offense_df1 = df_ytd.loc[(df_ytd['Oppt']==team)&(df_ytd['Week']==week)]
            offense_df2 = df_ytd.loc[(df_ytd['Oppt']==team)&(df_ytd['Week']==week+1)]
            sum_ = offense_df1['DK points'].sum()
            try:
                def_df.loc[(df_ytd['Team']==team)&(df_ytd['Week']==week+1), 'fantasy_points_allowed_lw'] = sum_
            except:
                print("couldn't append first sum")
                pass
            try:
                df_ytd.loc[(df_ytd['Oppt']==team)&(df_ytd['Week']==week+1), 'Oppt_pts_allowed_lw'] = sum_
            except:
                print("couldn't append second sum")
                pass
        except:
            pass

In [37]:
df_next_week['Oppt_pts_allowed_lw'] = 0
def_teams = [x for x in def_df['Team'].unique()]

for team in def_teams:
    try:
        offense_df1 = df_ytd.loc[(df_ytd['Oppt']==team)&(df_ytd['Week']==week)]
        sum_ = offense_df1['DK points'].sum()
        
        try:
            df_next_week.loc[(df_next_week['Oppt']==team), 'Oppt_pts_allowed_lw'] = sum_
        except:
            print("couldn't append second sum")
            pass
    except:
        pass

In [38]:
# Because we won't have access to some stats since we are trying to 
# project into the future, we'll need to be a little more creative.
# Instead of dropping 'DK points', substitute avg PPG for that value

def get_avg_ppg(ytd_df, player_name):
    filt = ytd_df['Name']==player_name
    working_df = ytd_df.loc[filt]
    mean = np.mean(working_df['DK points'])
    return mean

# def get_avg_ppg(ytd_df, player_name):
#     one_hot_columns = (ytd_df.iloc[:, 4:] == 1).idxmax(1)
#     ytd_df['player_name'] = one_hot_columns
#     ytd_df['player_name'] = ytd_df['player_name'].str.replace("Name_", "")
#     filt = ytd_df['player_name']==player_name
#     working_df = ytd_df.loc[filt]
#     mean = np.mean(working_df['DK points'])
#     return mean

# def get_avg_scoring_potential(ytd_df, player_name):
#     filt = ytd_df['Name']==player_name
#     working_df = ytd_df.loc[filt]
#     mean = round(np.mean(working_df['scoring_potential']),0)
#     return mean

if week == 1:
    df_next_week['DK points'] = 0
    for num in range(0,len(df_next_week)):
        df_next_week['DK points'][num] = get_avg_ppg(df_ytd, df_next_week['Name'][num])
else:
    df_next_week['DK points'] = 0
#     df_next_week['scoring_potential'] = 0
    for num in range(0,len(df_next_week)):
        df_next_week['DK points'][num] = get_avg_ppg(df_ytd, df_next_week['Name'][num])
#         df_next_week['scoring_potential'][num] = get_avg_scoring_potential(df_ytd, df_next_week['Name'][num])
df_next_week = df_next_week.fillna(0)
df_next_week

,Week,Name,Pos,Team,h/a,Oppt,DK points,DK salary,Oppt_pts_allowed_lw
0,8,"Mahomes II, Patrick",QB,kan,h,nyj,25.000000,8100,104.58
1,8,"Wilson, Russell",QB,sea,h,sfo,32.000000,7800,47.08
2,8,"Stafford, Matthew",QB,det,h,ind,18.000000,6400,0.00
3,8,"Rodgers, Aaron",QB,gnb,h,min,23.000000,7600,0.00
4,8,"Herbert, Justin",QB,lac,h,den,27.000000,6900,106.22
...,...,...,...,...,...,...,...,...,...
378,8,Detroit,Def,det,h,ind,2.500000,2500,0.00
379,8,Green Bay,Def,gnb,h,min,5.000000,2900,0.00
380,8,Tennessee,Def,ten,a,cin,4.500000,3000,134.44
381,8,San Francisco,Def,sfo,a,sea,6.714286,2700,147.30


In [39]:
# ordinal encode names, teams, h/a and oppts
df_ytd = ft_ordinal_encode_df(df_ytd, oe_names, oe_teams, oe_ha, oe_oppts)

In [40]:
# get dummies on the rest
df_ytd = pd.get_dummies(df_ytd)

In [41]:
df_ytd

,Week,Name,Team,h/a,Oppt,DK points,DK salary,scoring_potential,Oppt_pts_allowed_lw,Pos_Def,Pos_QB,Pos_RB,Pos_TE,Pos_WR
0,1,542.0,27.0,0.0,2.0,34.78,7000,3,0.00,0,1,0,0,0
1,1,425.0,11.0,0.0,21.0,33.76,6300,3,0.00,0,1,0,0,0
2,1,9.0,3.0,1.0,25.0,33.18,6500,3,0.00,0,1,0,0,0
3,1,432.0,1.0,1.0,28.0,27.90,6700,2,0.00,0,1,0,0,0
4,1,266.0,2.0,1.0,8.0,27.50,8100,2,0.00,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,7,371.0,21.0,1.0,5.0,1.00,3200,0,83.02,1,0,0,0,0
2858,7,445.0,27.0,0.0,1.0,0.00,2700,0,83.94,1,0,0,0,0
2859,7,93.0,6.0,1.0,8.0,-2.00,2800,0,100.42,1,0,0,0,0
2860,7,246.0,12.0,1.0,12.0,-2.00,2200,0,110.04,1,0,0,0,0


In [42]:
df_next_week = t_ordinal_encode_df(df_next_week, oe_names, oe_teams, oe_ha, oe_oppts)
df_next_week = find_scoring_potentials(df_next_week)
df_next_week = pd.get_dummies(df_next_week)

In [43]:
df_next_week

,Week,Name,Team,h/a,Oppt,DK points,DK salary,Oppt_pts_allowed_lw,scoring_potential,Pos_Def,Pos_QB,Pos_RB,Pos_TE,Pos_WR
0,8,326.0,15.0,1.0,25.0,25.000000,8100,104.58,2,0,1,0,0,0
1,8,542.0,27.0,1.0,29.0,32.000000,7800,47.08,3,0,1,0,0,0
2,8,475.0,10.0,1.0,14.0,18.000000,6400,0.00,1,0,1,0,0,0
3,8,425.0,11.0,1.0,21.0,23.000000,7600,0.00,2,0,1,0,0,0
4,8,221.0,16.0,1.0,10.0,27.000000,6900,106.22,2,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,8,127.0,10.0,1.0,14.0,2.500000,2500,0.00,0,1,0,0,0,0
379,8,196.0,11.0,1.0,21.0,5.000000,2900,0.00,0,1,0,0,0,0
380,8,497.0,30.0,0.0,7.0,4.500000,3000,134.44,0,1,0,0,0,0
381,8,438.0,28.0,0.0,28.0,6.714286,2700,147.30,0,1,0,0,0,0


In [44]:
X = df_ytd.drop(labels='scoring_potential', axis=1)
y = df_ytd['scoring_potential']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.2,
                                                    random_state=0)

In [45]:
# scale data for models that need scaling
scaled_X_train, scaled_X_test = scale_features(sc_salary, sc_points, sc_pts_ald, X_train, X_test, first_time=True)

In [46]:
svm_clf = SVC(kernel = 'linear', random_state = 0)
nb_clf = GaussianNB()
dt_clf = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
# grad_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1, max_depth=2, random_state=0)
grad_clf = GradientBoostingClassifier()
xgb_clf = XGBClassifier()

In [47]:
# SVM
svm_clf.fit(scaled_X_train, y_train)
y_pred = svm_clf.predict(scaled_X_test)
cm, acc_score = make_confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("Confusion Matrix: \n")
print(cm)
print("\n")
print("Accuracy: \n")
print(acc_score)
print("\n")
print("F1: \n")
print(f1)

Confusion Matrix: 

[[394   4   0   0]
 [ 11  99   1   0]
 [  0  11  33   0]
 [  0   0  12   8]]


Accuracy: 

0.9319371727748691


F1: 

0.9283431377327956


In [48]:
# Naive Bayes
nb_clf.fit(scaled_X_train, y_train)
y_pred = nb_clf.predict(scaled_X_test)
cm, acc_score = make_confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("Confusion Matrix: \n")
print(cm)
print("\n")
print("Accuracy: \n")
print(acc_score)
print("\n")
print("F1: \n")
print(f1)

Confusion Matrix: 

[[381  17   0   0]
 [  6  96   9   0]
 [  0   3  30  11]
 [  0   0   1  19]]


Accuracy: 

0.9179755671902269


F1: 

0.9194640270276131


In [49]:
# Decision Tree
dt_clf.fit(scaled_X_train, y_train)
y_pred = dt_clf.predict(scaled_X_test)
cm, acc_score = make_confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("Confusion Matrix: \n")
print(cm)
print("\n")
print("Accuracy: \n")
print(acc_score)
print("\n")
print("F1: \n")
print(f1)

Confusion Matrix: 

[[398   0   0   0]
 [  0 111   0   0]
 [  0   0  44   0]
 [  0   0   0  20]]


Accuracy: 

1.0


F1: 

1.0


In [50]:
# Gradient Boost
grad_clf.fit(scaled_X_train, y_train)
y_pred = grad_clf.predict(scaled_X_test)
cm, acc_score = make_confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("Confusion Matrix: \n")
print(cm)
print("\n")
print("Accuracy: \n")
print(acc_score)
print("\n")
print("F1: \n")
print(f1)

Confusion Matrix: 

[[398   0   0   0]
 [  0 111   0   0]
 [  0   0  44   0]
 [  0   0   0  20]]


Accuracy: 

1.0


F1: 

1.0


In [51]:
# XGBoost
xgb_clf.fit(scaled_X_train, y_train)
y_pred = xgb_clf.predict(scaled_X_test)
cm, acc_score = make_confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print("Confusion Matrix: \n")
print(cm)
print("\n")
print("Accuracy: \n")
print(acc_score)
print("\n")
print("F1: \n")
print(f1)

[10:44:53] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Confusion Matrix: 

[[398   0   0   0]
 [  0 111   0   0]
 [  0   0  44   0]
 [  0   0   0  20]]


Accuracy: 

1.0


F1: 

1.0


C:\Users\maxw2\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


So it's interesting here that with training the models the boosted methods and Decision trees continue to have perfect scores, which I am guessing means there's some overfitting going on. So I'd say those ones, we can safely ignore.

That leaves us with just 2 of the non-boosted methods (SVM & Naive Bayes).

In [52]:
df_next_week

,Week,Name,Team,h/a,Oppt,DK points,DK salary,Oppt_pts_allowed_lw,scoring_potential,Pos_Def,Pos_QB,Pos_RB,Pos_TE,Pos_WR
0,8,326.0,15.0,1.0,25.0,25.000000,8100,104.58,2,0,1,0,0,0
1,8,542.0,27.0,1.0,29.0,32.000000,7800,47.08,3,0,1,0,0,0
2,8,475.0,10.0,1.0,14.0,18.000000,6400,0.00,1,0,1,0,0,0
3,8,425.0,11.0,1.0,21.0,23.000000,7600,0.00,2,0,1,0,0,0
4,8,221.0,16.0,1.0,10.0,27.000000,6900,106.22,2,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,8,127.0,10.0,1.0,14.0,2.500000,2500,0.00,0,1,0,0,0,0
379,8,196.0,11.0,1.0,21.0,5.000000,2900,0.00,0,1,0,0,0,0
380,8,497.0,30.0,0.0,7.0,4.500000,3000,134.44,0,1,0,0,0,0
381,8,438.0,28.0,0.0,28.0,6.714286,2700,147.30,0,1,0,0,0,0


In [53]:
X_test = df_next_week.copy()
x_train_dumy, X_test_scaled = scale_features(sc_salary, sc_points, sc_pts_ald,X_train,X_test)
y_test_scaled = X_test_scaled['scoring_potential']
X_test_scaled = X_test_scaled.drop(labels="scoring_potential", axis=1)
X_test_scaled

,Week,Name,Team,h/a,Oppt,DK points,DK salary,Oppt_pts_allowed_lw,Pos_Def,Pos_QB,Pos_RB,Pos_TE,Pos_WR
0,8,326.0,15.0,1.0,25.0,0.483333,0.81,0.598352,0,1,0,0,0
1,8,542.0,27.0,1.0,29.0,0.600000,0.78,0.269367,0,1,0,0,0
2,8,475.0,10.0,1.0,14.0,0.366667,0.64,0.000000,0,1,0,0,0
3,8,425.0,11.0,1.0,21.0,0.450000,0.76,0.000000,0,1,0,0,0
4,8,221.0,16.0,1.0,10.0,0.516667,0.69,0.607735,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,8,127.0,10.0,1.0,14.0,0.108333,0.25,0.000000,1,0,0,0,0
379,8,196.0,11.0,1.0,21.0,0.150000,0.29,0.000000,1,0,0,0,0
380,8,497.0,30.0,0.0,7.0,0.141667,0.30,0.769196,1,0,0,0,0
381,8,438.0,28.0,0.0,28.0,0.178571,0.27,0.842774,1,0,0,0,0


In [54]:
# use svm_clf 
y_pred = svm_clf.predict(X_test_scaled)

In [55]:
df_check_results = get_weekly_data(next_week, season).drop(['Unnamed: 0', 'Year'], axis=1)

In [56]:
y_true = np.array(find_scoring_potentials(df_check_results)[['scoring_potential']]).flatten()
y_true

array([3, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [57]:
y_pred

array([2, 3, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 2, 1,
       2, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 2, 1, 1, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,

In [58]:
cm, acc_score = make_confusion_matrix(y_true, y_pred)
cm

array([[234,  37,   2,   0],
       [ 36,  34,   9,   0],
       [ 11,  11,   4,   1],
       [  0,   1,   3,   0]], dtype=int64)

In [59]:
acc_score

0.7101827676240209

:Sad Face Emoji: Right away, the thing that stands out is a sizeable drop in performance, which I'd say is most likely due to a drop in training set size.

So now let's check the other models and see how they do. I'd expect a similar result as with the SVM model.

In [60]:
y_pred_nb = nb_clf.predict(X_test_scaled)
y_pred_dt = dt_clf.predict(X_test_scaled)
y_pred_grad = grad_clf.predict(X_test_scaled)
y_pred_xgb = xgb_clf.predict(X_test_scaled)

cm_nb, acc_score_nb = make_confusion_matrix(y_true, y_pred_nb)
cm_dt, acc_score_dt = make_confusion_matrix(y_true, y_pred_dt)
cm_grad, acc_score_grad = make_confusion_matrix(y_true, y_pred_grad)
cm_xgb, acc_score_xgb = make_confusion_matrix(y_true, y_pred_xgb)

cms = [cm_nb, cm_dt, cm_grad, cm_xgb]
accs = [acc_score_nb, acc_score_dt, acc_score_grad, acc_score_xgb]

model = ["NB", "Decision Tree", "Gradient Boost", "XG Boost"]

In [61]:
print(f"season: {season}")
print(f"training week: {week}")
print(f"predicting week: {next_week}")
print('==============')
for i in range(0,4):
    print('Model: '+ model[i])
    print('CM: ')
    print(cms[i])
    print('Acc: ')
    print(round(accs[i]*100, 4))
    print('==============')

season: 2020
training week: 7
predicting week: 8
Model: NB
CM: 
[[232  38   2   1]
 [ 38  25  13   3]
 [ 10  11   2   4]
 [  0   1   1   2]]
Acc: 
68.1462
Model: Decision Tree
CM: 
[[229  41   3   0]
 [ 35  34  10   0]
 [ 10  12   4   1]
 [  0   1   3   0]]
Acc: 
69.7128
Model: Gradient Boost
CM: 
[[229  41   3   0]
 [ 35  34  10   0]
 [ 10  12   4   1]
 [  0   1   3   0]]
Acc: 
69.7128
Model: XG Boost
CM: 
[[229  41   3   0]
 [ 35  34  10   0]
 [ 10  12   4   1]
 [  0   1   3   0]]
Acc: 
69.7128


These results are pretty interesting. First, it appears that I was right about overfitting. The boosted models and decision tree have the worse accuracies, as far as correct predictions of for class 3 players.

Naive Bayes and SVM appear to be very accurate overall, but that seems to be largely due to a larger number of correct 0 predictions.

Taking a look at our criteria from the top of this page:

1. Correct 3 predictions - NB
2. Correct 0 predictions - All of them, with slight lead to SVM
3. Bottom right 2x2 has most counts - All equal, slight edge to NB
4. Minimize top row (not including top left) - SVM
5. Minimize left column (not including top left) - NB

There appears to be a bit of a tie between Naive Bayes and SVM. Since Naive Bayes is the only one that has correct 3 predictions, we'll use that one.

[Try again](#test_run)

## Build Player Pools Based on Results

In [62]:
X_test

,Week,Name,Team,h/a,Oppt,DK points,DK salary,Oppt_pts_allowed_lw,scoring_potential,Pos_Def,Pos_QB,Pos_RB,Pos_TE,Pos_WR
0,8,326.0,15.0,1.0,25.0,0.483333,0.81,0.598352,2,0,1,0,0,0
1,8,542.0,27.0,1.0,29.0,0.600000,0.78,0.269367,3,0,1,0,0,0
2,8,475.0,10.0,1.0,14.0,0.366667,0.64,0.000000,1,0,1,0,0,0
3,8,425.0,11.0,1.0,21.0,0.450000,0.76,0.000000,2,0,1,0,0,0
4,8,221.0,16.0,1.0,10.0,0.516667,0.69,0.607735,2,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,8,127.0,10.0,1.0,14.0,0.108333,0.25,0.000000,0,1,0,0,0,0
379,8,196.0,11.0,1.0,21.0,0.150000,0.29,0.000000,0,1,0,0,0,0
380,8,497.0,30.0,0.0,7.0,0.141667,0.30,0.769196,0,1,0,0,0,0
381,8,438.0,28.0,0.0,28.0,0.178571,0.27,0.842774,0,1,0,0,0,0


In [63]:
y_pred

array([2, 3, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 2, 1,
       2, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 2, 1, 1, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,

In [64]:
y_true

array([3, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [65]:
X_test_scaled.rename(columns={'scoring_potential': 'est_scoring_pot', 'DK points': 'avg_points'}, inplace=True)
# X_test_scaled['pred_scoring_pot'] = y_pred
X_test_scaled['pred_scoring_pot'] = y_pred_nb
# X_test_scaled['pred_scoring_pot'] = y_pred_dt
# X_test_scaled['pred_scoring_pot'] = y_pred_grad
# X_test_scaled['pred_scoring_pot'] = y_pred_xgb
X_test_scaled['act_scoring_pot'] = y_true
X_test_scaled['act_pts_scored'] = df_check_results['DK points']
X_test_scaled

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,Pos_Def,Pos_QB,Pos_RB,Pos_TE,Pos_WR,pred_scoring_pot,act_scoring_pot,act_pts_scored
0,8,326.0,15.0,1.0,25.0,0.483333,0.81,0.598352,0,1,0,0,0,3,3,39.64
1,8,542.0,27.0,1.0,29.0,0.600000,0.78,0.269367,0,1,0,0,0,3,2,28.74
2,8,475.0,10.0,1.0,14.0,0.366667,0.64,0.000000,0,1,0,0,0,2,2,27.44
3,8,425.0,11.0,1.0,21.0,0.450000,0.76,0.000000,0,1,0,0,0,3,2,23.54
4,8,221.0,16.0,1.0,10.0,0.516667,0.69,0.607735,0,1,0,0,0,3,2,23.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,8,127.0,10.0,1.0,14.0,0.108333,0.25,0.000000,1,0,0,0,0,0,0,0.00
379,8,196.0,11.0,1.0,21.0,0.150000,0.29,0.000000,1,0,0,0,0,0,0,0.00
380,8,497.0,30.0,0.0,7.0,0.141667,0.30,0.769196,1,0,0,0,0,0,0,-1.00
381,8,438.0,28.0,0.0,28.0,0.178571,0.27,0.842774,1,0,0,0,0,0,0,-2.00


In [66]:
# Wanting to print entire dataframes from here on
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [67]:
def invert_one_hot_encode(df, col_idx_start, col_idx_stop, sub_str):
    one_hot_columns = (df.iloc[:, col_idx_start:col_idx_stop] == 1).idxmax(1)
    df[sub_str] = one_hot_columns
    df[sub_str] = df[sub_str].str.replace(sub_str, "")
    df = df.drop(labels=one_hot_columns, axis=1)
    return df

X_test = invert_one_hot_encode(X_test_scaled, 8, 13, 'Pos_')
X_test.rename(columns={'Pos_': 'Pos'}, inplace=True)
X_test = it_ordinal_encode_df(X_test, oe_names, oe_teams, oe_ha, oe_oppts)
X_dummy, X_test = unscale_features(sc_salary, sc_points, sc_pts_ald, X_train, X_test)
X_test

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
0,8,"Mahomes II, Patrick",kan,h,nyj,25.000000,8100.0,104.58,3,3,39.64,QB
1,8,"Wilson, Russell",sea,h,sfo,32.000000,7800.0,47.08,3,2,28.74,QB
2,8,"Stafford, Matthew",det,h,ind,18.000000,6400.0,0.00,2,2,27.44,QB
3,8,"Rodgers, Aaron",gnb,h,min,23.000000,7600.0,0.00,3,2,23.54,QB
4,8,"Herbert, Justin",lac,h,den,27.000000,6900.0,106.22,3,2,23.22,QB
5,8,"Rivers, Philip",ind,a,det,15.000000,5900.0,114.92,1,2,22.28,QB
6,8,"Lock, Drew",den,h,lac,8.000000,5300.0,89.22,1,2,20.82,QB
7,8,"Brees, Drew",nor,a,chi,20.000000,6300.0,109.76,2,1,19.60,QB
8,8,"Newton, Cam",nwe,a,buf,19.000000,5700.0,47.70,2,1,19.36,QB
9,8,"Brady, Tom",tam,a,nyg,23.000000,6700.0,121.86,2,1,19.06,QB


In [68]:
X_test.loc[X_test.Pos=='QB']

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
0,8,"Mahomes II, Patrick",kan,h,nyj,25.0,8100.0,104.58,3,3,39.64,QB
1,8,"Wilson, Russell",sea,h,sfo,32.0,7800.0,47.08,3,2,28.74,QB
2,8,"Stafford, Matthew",det,h,ind,18.0,6400.0,0.00,2,2,27.44,QB
3,8,"Rodgers, Aaron",gnb,h,min,23.0,7600.0,0.00,3,2,23.54,QB
4,8,"Herbert, Justin",lac,h,den,27.0,6900.0,106.22,3,2,23.22,QB
5,8,"Rivers, Philip",ind,a,det,15.0,5900.0,114.92,1,2,22.28,QB
6,8,"Lock, Drew",den,h,lac,8.0,5300.0,89.22,1,2,20.82,QB
7,8,"Brees, Drew",nor,a,chi,20.0,6300.0,109.76,2,1,19.60,QB
8,8,"Newton, Cam",nwe,a,buf,19.0,5700.0,47.70,2,1,19.36,QB
9,8,"Brady, Tom",tam,a,nyg,23.0,6700.0,121.86,2,1,19.06,QB


In [69]:
X_test.loc[X_test.Pos=='RB']

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
34,8,"Cook, Dalvin",min,a,gnb,24.060000,7500.0,100.16,2,3,51.6,RB
35,8,"Kamara, Alvin",nor,a,chi,28.900000,8200.0,109.76,3,2,25.3,RB
36,8,"Bernard, Giovani",cin,h,ten,8.357143,5800.0,103.92,0,2,22.8,RB
37,8,"Dallas, DeeJay",sea,h,sfo,2.700000,4000.0,47.08,0,2,22.8,RB
38,8,"Hines, Nyheim",ind,a,det,9.616667,4000.0,114.92,0,2,21.2,RB
39,8,"Wilkins, Jordan",ind,a,det,2.266667,4000.0,114.92,0,2,20.3,RB
40,8,"Henry, Derrick",ten,a,cin,22.566667,8000.0,134.44,2,2,20.2,RB
41,8,"Moss, Zack",buf,h,nwe,6.650000,4400.0,123.48,0,2,20.1,RB
42,8,"Harris, Damien",nwe,a,buf,8.433333,4200.0,47.70,0,1,19.2,RB
43,8,"Williams, Jamaal",gnb,h,min,10.416667,6100.0,0.00,1,1,18.2,RB


In [70]:
X_test.loc[X_test.Pos=='WR']

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
132,8,"Metcalf, D.K.",sea,h,sfo,18.483333,7500.0,47.08,1,3,43.10,WR
133,8,"Adams, Davante",gnb,h,min,27.725000,8800.0,0.00,3,3,30.30,WR
134,8,"Davis, Corey",ten,a,cin,15.025000,5100.0,134.44,1,2,29.80,WR
135,8,"Woods, Robert",lar,a,mia,14.414286,6300.0,0.00,1,2,28.40,WR
136,8,"Hill, Tyreek",kan,h,nyj,16.614286,6700.0,104.58,1,2,25.80,WR
137,8,"Kupp, Cooper",lar,a,mia,13.714286,6500.0,0.00,1,2,25.00,WR
138,8,"Jones, Julio",atl,a,car,18.740000,7000.0,109.98,1,2,23.70,WR
139,8,"Aiyuk, Brandon",sfo,a,sea,12.650000,5800.0,147.30,1,2,23.10,WR
140,8,"Hardman, Mecole",kan,h,nyj,7.957143,4700.0,104.58,0,2,22.60,WR
141,8,"Allen, Keenan",lac,h,den,18.183333,6200.0,106.22,1,2,21.70,WR


In [71]:
X_test.loc[X_test.Pos=='TE']

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
272,8,"Kelce, Travis",kan,h,nyj,17.442857,6600.0,104.58,1,2,27.9,TE
273,8,"Cook, Jared",nor,a,chi,10.360000,4400.0,109.76,0,1,16.1,TE
274,8,"Ebron, Eric",pit,a,bal,8.083333,3900.0,0.00,0,1,14.8,TE
275,8,"Gronkowski, Rob",tam,a,nyg,8.857143,4900.0,121.86,0,1,14.1,TE
276,8,"Hockenson, T.J.",det,h,ind,12.266667,5300.0,0.00,1,1,13.5,TE
277,8,"Tonyan, Robert",gnb,h,min,11.833333,4300.0,0.00,1,1,12.9,TE
278,8,"Engram, Evan",nyg,h,tam,7.928571,4400.0,89.26,0,1,12.0,TE
279,8,"Fant, Noah",den,h,lac,12.340000,4700.0,89.22,1,1,11.7,TE
280,8,"Schultz, Dalton",dal,a,phi,9.171429,3700.0,90.18,0,1,11.3,TE
281,8,"Hurst, Hayden",atl,a,car,10.357143,4200.0,109.98,0,1,10.4,TE


In [72]:
# explanation for this is below
def_df = X_test.loc[X_test.Pos=='Def']
def_df

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
355,8,Miami,mia,h,lar,8.000000,2400.0,78.44,0,2,23.0,Def
356,8,Philadelphia,phi,h,dal,5.714286,3500.0,108.96,0,2,20.0,Def
357,8,Pittsburgh,pit,a,bal,9.333333,3600.0,0.00,0,1,18.0,Def
358,8,Indianapolis,ind,a,det,11.166667,3100.0,114.92,0,1,15.0,Def
359,8,Dallas,dal,a,phi,1.285714,2400.0,90.18,0,1,13.0,Def
360,8,Las Vegas,lvr,a,cle,0.833333,2900.0,141.18,0,0,9.0,Def
361,8,Kansas City,kan,h,nyj,9.285714,4500.0,104.58,0,0,9.0,Def
362,8,Tampa Bay,tam,a,nyg,9.857143,4300.0,121.86,0,0,7.0,Def
363,8,New Orleans,nor,a,chi,4.333333,3400.0,109.76,0,0,7.0,Def
364,8,Seattle,sea,h,sfo,5.333333,3200.0,47.08,0,0,7.0,Def


In [73]:
df_for_lineups = X_test
df_for_lineups

,Week,Name,Team,h/a,Oppt,avg_points,DK salary,Oppt_pts_allowed_lw,pred_scoring_pot,act_scoring_pot,act_pts_scored,Pos
0,8,"Mahomes II, Patrick",kan,h,nyj,25.000000,8100.0,104.58,3,3,39.64,QB
1,8,"Wilson, Russell",sea,h,sfo,32.000000,7800.0,47.08,3,2,28.74,QB
2,8,"Stafford, Matthew",det,h,ind,18.000000,6400.0,0.00,2,2,27.44,QB
3,8,"Rodgers, Aaron",gnb,h,min,23.000000,7600.0,0.00,3,2,23.54,QB
4,8,"Herbert, Justin",lac,h,den,27.000000,6900.0,106.22,3,2,23.22,QB
5,8,"Rivers, Philip",ind,a,det,15.000000,5900.0,114.92,1,2,22.28,QB
6,8,"Lock, Drew",den,h,lac,8.000000,5300.0,89.22,1,2,20.82,QB
7,8,"Brees, Drew",nor,a,chi,20.000000,6300.0,109.76,2,1,19.60,QB
8,8,"Newton, Cam",nwe,a,buf,19.000000,5700.0,47.70,2,1,19.36,QB
9,8,"Brady, Tom",tam,a,nyg,23.000000,6700.0,121.86,2,1,19.06,QB


## Some observations...

The algorithm, so far, is pretty decent at picking everything except for defenses. So that'll need to be re-examined later, but for now, we'll just use all the defenses and see what happens.

## Build some lineups

<a id="lineup_builder"></a>

In [80]:
class Lineup:
    """ 
    takes the results of the model prediction (dataframe 
    with attached predictions) and builds out a few lineups 
    """
    def __init__(self, df, def_df, verbose=False):
        self.verbose = verbose
        self.df = df
        self.def_df = def_df
        self.current_salary = 0
        self.good_scoring_potential = False
        self.no_duplicates = False
        self.top_lineups = []
        self.qbs = []
        self.rbs = []
        self.wrs = []
        self.tes = []
        self.flex = []
        self.defs = []
    
    def find_top_10(self, position):
        arr = []
        end_of_range = len(self.df.loc[self.df['Pos']==position])
        if position == 'Flex':
            position_df = self.df.loc[(self.df['Pos']=='RB')|(self.df['Pos']=='TE')|(self.df['Pos']=='WR')]
            end_of_range = (len(self.df.loc[self.df['Pos']=='RB'])+
                            len(self.df.loc[self.df['Pos']=='WR'])+
                            len(self.df.loc[self.df['Pos']=='TE']))
        elif position == 'Def':
            end_of_range = len(self.def_df)
            position_df = self.def_df
            position_df = position_df.sort_values(by='avg_points', ascending=False)
        else:
            position_df = self.df.loc[self.df['Pos']==position]
        
        # print(position_df)
        for row in range(0,end_of_range):
            player = {
                'name': position_df.iloc[row]['Name'],
                'team': position_df.iloc[row]['Team'],
                'h/a': position_df.iloc[row]['h/a'],
                'pos': position_df.iloc[row]['Pos'],
                'salary': position_df.iloc[row]['DK salary'],
                'avg_points': position_df.iloc[row]['avg_points'],
                'scoring_pot': position_df.iloc[row]['pred_scoring_pot'],
                'act_pts':position_df.iloc[row]['act_pts_scored']
            }
            if len(arr) < end_of_range:
                arr.append(player)
            else: 
                break
        return arr
    
    def get_players(self):
        top_10_qbs = self.find_top_10(position='QB')
        top_10_rbs = self.find_top_10(position='RB')
        top_10_wrs = self.find_top_10(position='WR')
        top_10_tes = self.find_top_10(position='TE')
        top_10_flex = self.find_top_10(position='Flex')
        top_10_defs = self.find_top_10(position='Def')
        return top_10_qbs, top_10_rbs, top_10_wrs, top_10_tes, top_10_flex, top_10_defs
    
    def check_salary(self, lineup):
        current_salary = 0
        for keys in lineup.keys():
            current_salary += lineup[keys]['salary']
        return current_salary
    
    def check_duplicates(self, lineup):
        rb1_name = lineup['RB1']['name']
        rb2_name = lineup['RB2']['name']
        flex_name = lineup['Flex']['name']
        wr1_name = lineup['WR1']['name']
        wr2_name = lineup['WR2']['name']
        wr3_name = lineup['WR3']['name']
        te_name = lineup['TE']['name']
        names = [flex_name, rb1_name, rb2_name, wr1_name, wr2_name, wr3_name, te_name]
        while len(names) > 1:
            if names[0] in names[1:-1]:
                return False
            else:
                names.pop(0)   
        return True
    
    def check_scoring_potentials(self, lineup):
        qb_sp = lineup['QB']['scoring_pot']
        rb1_sp = lineup['RB1']['scoring_pot']
        rb2_sp = lineup['RB2']['scoring_pot']
        flex_sp = lineup['Flex']['scoring_pot']
        wr1_sp = lineup['WR1']['scoring_pot']
        wr2_sp = lineup['WR2']['scoring_pot']
        wr3_sp = lineup['WR3']['scoring_pot']
        te_sp = lineup['TE']['scoring_pot']
        def_sp = lineup['Def']['scoring_pot']
        scor_pots = [qb_sp, flex_sp, rb1_sp, rb2_sp, wr1_sp, wr2_sp, wr3_sp, te_sp, def_sp]
        if np.mean(scor_pots) < 0.75:
            return False
        else:
            return True
    
    def shuffle_players(self):
        lineup = {
            'QB': self.qbs[random.randrange(len(self.df.loc[self.df['Pos']=='QB']))],
            'RB1': self.rbs[random.randrange(len(self.df.loc[self.df['Pos']=='RB']))],
            'RB2': self.rbs[random.randrange(len(self.df.loc[self.df['Pos']=='RB']))],
            'WR1': self.wrs[random.randrange(len(self.df.loc[self.df['Pos']=='WR']))],
            'WR2': self.wrs[random.randrange(len(self.df.loc[self.df['Pos']=='WR']))],
            'WR3': self.wrs[random.randrange(len(self.df.loc[self.df['Pos']=='WR']))],
            'TE': self.tes[random.randrange(len(self.df.loc[self.df['Pos']=='TE']))],
            'Flex': self.flex[random.randrange(len(self.df.loc[self.df['Pos']=='RB'])+
                                               len(self.df.loc[self.df['Pos']=='WR'])+
                                               len(self.df.loc[self.df['Pos']=='TE']))],
            'Def': self.defs[random.randrange(len(self.def_df))]
        }
        return lineup
    
    def build_lineup(self):
        self.current_salary = 100*1000
        self.no_duplicates = False
        self.qbs, self.rbs, self.wrs, self.tes, self.flex, self.defs = self.get_players()
        lineup = {
            'QB': self.qbs[0],
            'RB1': self.rbs[0],
            'RB2': self.rbs[1],
            'WR1': self.wrs[0],
            'WR2': self.wrs[1],
            'WR3': self.wrs[2],
            'TE': self.tes[0],
            'Flex': self.flex[9], # started at the end of flex to avoid duplicating players
            'Def': self.defs[0]
        }
        # in theory, because of the legwork done by the algorithm,
        # any lineup should be good as long as it abides by the
        # constraints of DraftKings' team structures. So for
        # now, this will just give us the first 5 lineups that
        # fit within the salary cap and meet the other requirements
        
        while True:
            if self.verbose:
                print('======================')
                print(self.current_salary)
                print(self.no_duplicates)
                print(self.good_scoring_potential)
                print('======================')
            if (self.current_salary < 50*1000 
            and self.current_salary >= 45*1000 
            and self.no_duplicates
            and self.good_scoring_potential):
                break
            lineup = self.shuffle_players()
            # check scoring potential, making sure it averages to at least 1.25
            self.good_scoring_potential = self.check_scoring_potentials(lineup)
            # check salary, making sure it's between 45k and 50k
            self.current_salary = self.check_salary(lineup)
            # make sure there are no duplicate players
            self.no_duplicates = self.check_duplicates(lineup)
        
        self.top_lineups.append(lineup)
        print(f"added lineup. total lineups: {len(self.top_lineups)}")
    
lineup = Lineup(df_for_lineups, def_df)

In [81]:
# this step takes a while
for x in range (0,25):
    lineup.build_lineup()

added lineup. total lineups: 1
added lineup. total lineups: 2
added lineup. total lineups: 3
added lineup. total lineups: 4
added lineup. total lineups: 5
added lineup. total lineups: 6
added lineup. total lineups: 7
added lineup. total lineups: 8
added lineup. total lineups: 9
added lineup. total lineups: 10
added lineup. total lineups: 11
added lineup. total lineups: 12
added lineup. total lineups: 13
added lineup. total lineups: 14
added lineup. total lineups: 15
added lineup. total lineups: 16
added lineup. total lineups: 17
added lineup. total lineups: 18
added lineup. total lineups: 19
added lineup. total lineups: 20
added lineup. total lineups: 21
added lineup. total lineups: 22
added lineup. total lineups: 23
added lineup. total lineups: 24
added lineup. total lineups: 25


In [82]:
trash_count = 0
for line in lineup.top_lineups:
    lineup_df = pd.DataFrame.from_dict(line)
    if lineup_df.T['act_pts'].sum() < 120:
        trash_count += 1
        continue
    print(lineup_df.T)
    print('======================')
    print("Salary: " + str(lineup_df.T['salary'].sum()))
    print('======================')
    print("Pts: " + str(lineup_df.T['act_pts'].sum()))
    print('======================')
    print('======================')
    print('======================')
print("trash_count: " + str(trash_count))

                     name team h/a  pos  salary avg_points scoring_pot act_pts
QB    Mahomes II, Patrick  kan   h   QB  8100.0       25.0           3   39.64
RB1     Montgomery, David  chi   h   RB  5900.0  13.242857           1    12.5
RB2      Thompson, Darwin  kan   h   RB  4000.0       0.65           0     0.3
WR1         Byrd, Damiere  nwe   a   WR  3700.0   6.883333           0     6.9
WR2       Golladay, Kenny  det   h   WR  6600.0      17.95           1     0.0
WR3     Jefferson, Justin  min   a   WR  6500.0      18.45           1     5.6
TE        Toilolo, Levine  nyg   h   TE  2500.0   0.328571           0     1.9
Flex         Cook, Dalvin  min   a   RB  7500.0      24.06           2    51.6
Def           LA Chargers  lac   h  Def  4000.0   5.166667           0     3.0
Salary: 48800.0
Pts: 121.44
                   name team h/a  pos  salary avg_points scoring_pot act_pts
QB          Carr, Derek  lvr   a   QB  5500.0       21.0           2   12.58
RB1        Gurley, Todd  atl